<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/2023/ex15notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2023 ex15notebookA

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo15.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2023

In [ ]:
# いつものいろいろインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# SciPy の DCT と FFT の関数
from scipy.fft import dct, fft, ifft

# 科学技術計算のライブラリ SciPy の中の WAVE ファイルを扱うモジュール
from scipy.io.wavfile import read, write

---
## 離散フーリエ変換
---



前回，（正規）直交基底を用いてベクトルを展開・変換する直交展開について学びました．直交展開を用いると，データに含まれる成分を分析したり，データ圧縮・次元削減ができるのでした．また，直交展開の具体例として，「離散コサイン変換」（DCT, Discrete Cosine Transform）についても学びました．

今回は，ベクトルの直交展開の別の例として，**離散フーリエ変換** (DFT, Discrete Fourier Transform（よ）) について学びます．
DCTと同様に，画像や音声などのデータの分析や処理によく用いられます．


※ よだんだよん: フーリエ変換やフーリエ級数展開の Fourier は，フランスの数学者 Joseph Fourier (1768 - 1830) に由来します．Wikipedia の「[ジョゼフ・フーリエ](https://ja.wikipedia.org/wiki/%E3%82%B8%E3%83%A7%E3%82%BC%E3%83%95%E3%83%BB%E3%83%95%E3%83%BC%E3%83%AA%E3%82%A8)」．

---
### 準備

あとの実験で使う関数とデータを用意しておきます．

In [ ]:
# WAVE ファイルを読み込む関数
#
def readWAVE(filename):

    framerate, data = read(filename)

    # チャンネル数とフレーム数（データ点の数）を求める
    if data.ndim == 1:
        nchannels = 1
    else:
        nchannels = data.shape[1]
    nframes = data.shape[0]

    print('filename = ', filename)
    print('nchannels = ', nchannels)       # チャンネル数
    print('framerate = ', framerate)       # 標本化周波数
    print('nframes = ', nframes)             # フレーム数
    print('duration = {0:.2f}[sec]'.format(nframes / framerate))   # 長さ（秒単位）
    print('dtype = ', data.dtype)            # データ型（量子化ビット数に対応）

    assert data.dtype == 'uint8' or data.dtype == 'int16' or data.dtype == 'int32' or data.dtype == 'float32'

    # 最初の10秒分だけ取り出す（元がそれより短ければそのまま）
    nframesNew = min(framerate * 10, nframes)
    if nchannels == 1:
        dataNew = data[:nframesNew]
    else:
        # 多チャンネル信号なら0番目と1番目の平均値を取り出す
        if data.dtype == 'float32':  # 浮動小数点数のときは [0, 1] の値なので普通に足して2で割る
            dataNew = (data[:nframesNew, 0] + data[:nframesNew, 1])/2
        else: # 整数型のときはオーバーフローする可能性があるので，いったん64bit整数にしてから
            data64 = (data[:nframesNew, 0].astype(np.int64) + data[:nframesNew, 1].astype(np.int64))//2
            dataNew = data64.astype(data.dtype)

    return framerate, dataNew

In [ ]:
# WAVE ファイルを入手
#
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/Sound-Guitar4-ChordC.wav
fn = 'Sound-Guitar4-ChordC.wav'

import os

if not os.path.exists(fn):
    print(f'{fn}のダウンロードがうまくできていないようです')

In [ ]:
# WAVE ファイルを読み込む
#
framerate, dat = readWAVE(fn)
dat = dat.astype(float)
dat -= np.mean(dat)
guitar = dat

以下のリンクをたどると WAVE ファイルを自分のPCにダウンロードすることができます．音を聞いてみるとよいでしょう．

https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/Sound-Guitar4-ChordC.wav

---
### 離散フーリエ変換 (DFT, Discrete Fourier Transform)


離散コサイン変換の基底や展開係数は実数でできていましたが，離散フーリエ変換では基底も展開係数も複素数を含みます．

> **離散フーリエ変換**
>
> $N$ 次元ベクトル $\mathbf{u}_0, \mathbf{u}_1, \ldots, \mathbf{u}_{N-1}$ を次のように定めると，$\{\mathbf{u}_0, \mathbf{u}_1, \ldots, \mathbf{u}_{N-1}\}$ は直交基底となる．
>
> $$
\mathbf{u}_k = \left(  e^{0\times i\omega_0k}, e^{1\times i\omega_0k}, e^{2\times i\omega_0k}, \ldots, e^{(N-1)\times i\omega_0k} \right) \quad (k = 0, 1, 2, \ldots, N-1) \qquad (9)
$$
>
> ただし，$i = \sqrt{-1}, w_0 = \frac{2\pi}{N}$ である．
> $\{ \mathbf{u}_k \}$ を用いると，任意の $N$ 次元ベクトル $\mathbf{x} = (x_0, x_1, \ldots, x_{N-1})$ を
>
> $$
\mathbf{x} = c_0\mathbf{u}_0 + c_1\mathbf{u}_1 + \cdots + c_{N-1}\mathbf{u}_{N-1} \qquad (10)
$$
>
> と一意に展開できる．展開係数 $c_k$ の値は次式で求められる．
>
> $$
c_k = \frac{1}{N}\mathbf{x}\cdot\mathbf{u}_k = \frac{1}{N}\sum_{n=0}^{N-1} x_{n}e^{-i\omega_0kn} \qquad (11)
$$
>
> $\mathbf{x}$ から $\mathbf{c} = (c_0, c_1, \ldots, c_{N-1})$ を求める演算を **離散フーリエ変換** という．また，展開係数のことを **フーリエ係数** という．

いろいろわけわかめなところがあると思いますので，説明していきます．

まず，この基底を構成するベクトルがどんなものなのかを理解するために，値を眺めたりグラフに描いたりしてみましょう．ここでは，[SciPy](https://scipy.org/) の関数である [`scipy.fft.fft`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fft.fft.html) を使っています．

$N = 4$ のときは， $\omega_0 = \frac{2\pi}{4} = \frac{\pi}{2}$ で，

$$
\begin{aligned}
\mathbf{u}_0 &= \left( e^{0\times\frac{0\pi}{2}i}, e^{1\times\frac{0\pi}{2}i}, e^{2\times\frac{0\pi}{2}i}, e^{3\times\frac{0\pi}{2}i} \right) = (1, 1, 1, 1)\\
\mathbf{u}_1 &= \left( e^{0\times\frac{\pi}{2}i}, e^{1\times\frac{\pi}{2}i}, e^{2\times\frac{\pi}{2}i}, e^{3\times\frac{\pi}{2}i} \right) = (1, i, -1, -i)\\
\mathbf{u}_2 &= \left( e^{0\times\frac{2\pi}{2}i}, e^{1\times\frac{2\pi}{2}i}, e^{2\times\frac{2\pi}{2}i}, e^{3\times\frac{2\pi}{2}i} \right) = (1, -1, 1, -1)\\
\mathbf{u}_3 &= \left( e^{0\times\frac{3\pi}{2}i}, e^{1\times\frac{3\pi}{2}i}, e^{2\times\frac{3\pi}{2}i}, e^{3\times\frac{3\pi}{2}i} \right) = (1, -i, -1, i)\\
\end{aligned}
$$

となります．

In [ ]:
# 4次元の DFT の基底
U_dft4 = np.conj(fft(np.eye(4)))
U_dft4

NumPy では，虚数単位を `j` で表します．上記の最初の列が $\mathbf{u}_0$ に対応しています．

次に，$N=16$ の基底の一部をグラフに描いてみましょう．

In [ ]:
# 16次元の DFT の基底のうち最初の4つ
N = 16
U_dft = np.conj(fft(np.eye(N)))
fig, ax = plt.subplots(2, 4, figsize=(16, 6))
xp = np.linspace(0, N-1, num=200)
for k in range(4):
    # 実部
    re = U_dft[:, k].real # 実部
    ax[0][k].plot(re, 'o', color='blue', label=f'Re u{k}')
    omega0 = 2*np.pi/N
    yp = np.cos(omega0*k*xp)
    ax[0][k].plot(xp, yp, color='gray')
    # 虚部
    im = U_dft[:, k].imag # 虚部
    ax[1][k].plot(im, 'o', color='red', label=f'Im u{k}')
    yp = np.sin(omega0*k*xp)
    ax[1][k].plot(xp, yp, color='gray')
    for i in range(2):
        ax[i][k].set_ylim(-1.4, 1.4)
        ax[i][k].axhline(0, color='gray')
        ax[i][k].legend()
plt.show()

<img width=500 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/Euler.png" align="right">

上段が実部，下段が虚部で，左から順に $\mathbf{u}_0$, $\mathbf{u}_1$, $\mathbf{u}_2$, $\mathbf{u}_3$ を表しています．実部は $\cos$，虚部は $\sin$ の形をしています．このことは，オイラーの等式を用いて基底の式を変形してみると分かります（右図も参照）．

$\mathbf{u}_1$ の各要素は，複素平面上の単位円周上を $1$ からスタートして周期 $N$ で回転する点の座標を間隔$1$でサンプリングしたものとなっており，$\mathbf{u}_2$ の各要素は同様に周期 $N/2$ で（つまり2倍の速さで）回転する点の座標，$\mathbf{u}_3$ の各要素は周期 $N/3$ で回転する点の座標，...，というようになっています（よ）．つまり，番号の大きい基底ほど，対応する波の周期が短く（周波数が高く）なっています．

ただし，上記のことがいえるのは基底の番号 $k$ が $1$ 以上 $\frac{N}{2}$ 以下の場合です．$\frac{N}{2} < k \leq N-1$ のときは，$\mathbf{u}_k$ の各要素は周期 $N/(N-k)$ で単位円周上を逆向きに回転します（下図参照）．

離散コサイン変換の場合と同様に，これらの基底を用いた直交展開の展開係数（フーリエ係数）の値を調べることで，元のベクトルの中に様々な波の成分がどれくらい含まれているのかを知ることができます．


※よだんだよん: 複素平面上に $1$ を頂点に持つ正$N$角形を考えると，$\mathbf{u}_1$ の要素の値は，$1$ からスタートして反時計周りに一つずつ頂点を移動していったときの頂点の座標となっています．$\mathbf{u}_2$ は1つ飛ばし，$\mathbf{u}_3$ は2つ飛ばし...．


In [ ]:
# 16次元の DFT の基底のうち最後の4つ
N = 16
U_dft = np.conj(fft(np.eye(N)))
fig, ax = plt.subplots(2, 4, figsize=(16, 6))
xp = np.linspace(0, N-1, num=200)
for k in range(4):
    k2 = k + 12
    # 実部
    re = U_dft[:, k2].real # 実部
    ax[0][k].plot(re, 'o', color='blue', label=f'Re u{k2}')
    omega0 = 2*np.pi/N
    yp = np.cos(omega0*(k2-N)*xp)
    ax[0][k].plot(xp, yp, color='gray')
    # 虚部
    im = U_dft[:, k2].imag # 虚部
    ax[1][k].plot(im, 'o', color='red', label=f'Im u{k2}')
    yp = np.sin(omega0*(k2-N)*xp)
    ax[1][k].plot(xp, yp, color='gray')
    for i in range(2):
        ax[i][k].set_ylim(-1.4, 1.4)
        ax[i][k].axhline(0, color='gray')
        ax[i][k].legend()
plt.show()

ところで，注意深く式を眺めたひとはお気づきかもしれませんが，式$(11)$の $e$ の肩に乗っている式には負号がついています．これだと $\mathbf{u}_k\cdot\mathbf{x}$ と等しくなくておかしいように見えます．しかし，式$(11)$は正しい式です．実は，複素数を要素にもつベクトル $\mathbf{a} = (a_1, a_2, \ldots, a_D)$ と $\mathbf{b} = (b_1, b_2, \ldots, b_D)$ の内積は，次のように定義されます（注）．

$$
\mathbf{a}\cdot\mathbf{b} = \sum_{d=1}^D a_d \overline{{b}_d}
$$

$\overline{{b}_d}$ は，$b_d$ の複素共役です．そのため，式$(11)$ に $\overline{e^{i\omega_0kn}} = e^{-i\omega_0kn}$ が現れています．

※ 注意: この定義から，$\mathbf{a}\cdot\mathbf{b} = \overline{\mathbf{b}\cdot\mathbf{a}}$ が導かれます．$\mathbf{a}, \mathbf{b}$ が実ベクトルならば成り立つ $\mathbf{a}\cdot\mathbf{b} = \mathbf{b}\cdot\mathbf{a}$ という性質が，複素数ベクトルでは成り立ちません．

4次元のDFTの基底を例にして，これらが上記の内積の定義のもとで直交基底となっていることを確認してみます．

In [ ]:
# 4次元の DFT の基底
U_dft4

In [ ]:
U_dft4 @ np.conj(U_dft4)

上のセルの `np.conj` は，引数で渡された np.array の複素共役の値を返す NumPy の関数です． 対角要素以外が $0$ になっていることから，`U_dft4` の各列が互いに直交していることが分かります（注）．

※注意: 式$(9)$のベクトルたちは互いに直交してはいますが大きさは $1$ ではなく $N$ ですので，正規直交基底ではない直交基底です．全ての値を $\frac{1}{\sqrt{N}}$ すれば正規直交基底となりますが，離散フーリエ変換の定義はここで説明している形のものが一般的ですので，ここでもあえて正規直交基底にはしていません．

---
### 基底・係数の対称性と振幅・位相

以前も使ったことのある $N=16$ 次元のベクトルの例で，実際に離散フーリエ変換やってみましょう．

In [ ]:
x16 = np.array([0, -2, 0, -4, 10, 12, 10, 12, 8, 5, 4, 3, -2, 0, 2, 0])
print(x16)
fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(x16, '-o', color='red', linewidth=2)
ax.axhline(0, color='gray')
ax.set_ylim(-5, 15)
plt.show()

[`scipy.fft.fft`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fft.fft.html) に np.array を渡すとフーリエ係数が返ってきます．

In [ ]:
# x16 の離散フーリエ変換
C16 = fft(x16)

# 上記は次の計算と等価だがより計算量の少ないアルゴリズムを用いているので，
# N が大きい場合は上記の方が圧倒的に速い
#C16 = np.conj(U_dft.T) @ x16

print(C16)

# フーリエ係数の可視化（実部と虚部）
fig, ax = plt.subplots(1, 2, figsize=(12, 4))
ax[0].stem(C16.real, markerfmt='o', label='Re Ck')
ax[0].set_ylim(-60, 60)
ax[0].legend()
ax[1].stem(C16.imag, markerfmt='o', label='Im Ck')
ax[1].set_ylim(-60, 60)
ax[1].legend()
plt.show()

このグラフを見ると，得られたフーリエ係数の値に対称性があることが分かります（実部のグラフは $c_0$ を除き $k = \frac{N}{2} = 8$ を軸として左右対称，虚部の方は符号を反転したうえで左右対称）．
前のセクションで，離散フーリエ変換の基底ベクトル $\mathbf{u}_k$ のうち，$\frac{N}{2} < k \leq N-1$ のものは，「逆向きに回転する」という説明をしました．このことを式で表すと，次のようになります（導出は省略します）．

$$
\mathbf{u}_{N-k} = \overline{\mathbf{u}_k} \qquad (k = 1, 2, \ldots, N-1)
$$

この性質からさらに，要素が実数のベクトル $\mathbf{x}$ の
フーリエ係数には

$$
c_{N - k} = \overline{c_{k}} \qquad (k = 1, 2, \ldots, N-1)
$$

という対称性があることが導かれます（これも導出は省略します）．上記のグラフの対称性は，これを反映したものです．





一方，ある複素数 $z$ の偏角が $\arg z = \phi$ であるとき，$z = |z|e^{i\phi}$ より $\overline{z} = |z|e^{-i\phi}$ と表せます．ゆえに，

$$
\begin{aligned}
ze^{i\theta} + \overline{z}e^{-i\theta} &= |z|e^{i(\theta+\phi)} + |z|e^{-(\theta+\phi)} = |z|(e^{\theta+\phi} + e^{-(\theta+\phi)} )\\
&= 2|z|\cos{(\theta+\phi)}
\end{aligned}
$$

と変形できます（最後の等号は，オイラーの等式より導かれる関係式 $\frac{e^{i\theta}+e^{-i\theta}}{2} = \cos\theta$ より）．

以上のことを用いると，$\mathbf{u}_k$ の要素と $c_k$ について，

$$
\begin{aligned}
c_ke^{i\omega_0kn} + c_{N-k}e^{i\omega_0(N-k)n} &= c_ke^{i\omega_0kn} + \overline{c_{k}}\overline{e^{i\omega_0kn}} = c_ke^{i\omega_0kn} + \overline{c_{k}}e^{-i\omega_0kn}\\
&= 2|c_k|\cos{(\omega_0kn+\arg c_k)} \qquad (k = 1, 2, \ldots, N-1, n = 1, 2, \ldots, N-1)
\end{aligned}
$$

と表せます．

したがって，$c_k\mathbf{u}_k + c_{N-k}\mathbf{u}_{N-k}$ は，「振幅 $2|c_k|$ ，位相 $\arg c_k$ で周期 $N/k$ の $\cos$」を表すことになります．つまり，離散フーリエ変換は，与えられたベクトル $\mathbf{x}$ を，

$$
\begin{array}{ll}
\mbox{定数:} & c_0\mathbf{u}_0\\
\mbox{周期 $N$ の波:} & c_1\mathbf{u}_1 + c_{N-1}\mathbf{u}_{N-1} \\
\mbox{周期 $N/2$ の波:} & c_2\mathbf{u}_2 + c_{N-2}\mathbf{u}_{N-2} \\
\mbox{周期 $N/3$ の波:} & c_3\mathbf{u}_3 + c_{N-3}\mathbf{u}_{N-3}\\
: & :
\end{array}
$$

の和として表しているといえます．

次のセルを実行すると，`x16` に格納された $N=16$ 次元ベクトル $\mathbf{x}$ から求めたフーリエ係数 $c_0, c_1, c_2, \ldots, c_{15}$ を用いて

$$
\mathbf{z} = c_0\mathbf{u}_0 + (c_1\mathbf{u}_1 + c_{N-1}\mathbf{u}_{N-1}) + (c_2\mathbf{u}_2 + c_{N-2}\mathbf{u}_{N-2}) + (c_3\mathbf{u}_3 + c_{N-3}\mathbf{u}_{N-3})
$$

という $16$ 次元ベクトルを求めた結果を図示します．

In [ ]:
# x16 を k = 0, 1, 2, 3, 13, 14, 15 の成分のみで近似

N = 16
U_dft = np.conj(fft(np.eye(N)))
C16 = fft(x16)

C16_truncated = np.copy(C16)
C16_truncated[3+1:N-3] = 0
z = ifft(C16_truncated).real
print(z)

fig = plt.figure(figsize=(8, 16))

ax = fig.add_subplot(5, 1, 1)
ax.plot(x16, '-o', color='red', label='x16')
ax.plot(z, '-o', color='blue', label='z')
ax.axhline(0, color='gray')
ax.set_ylim(-10, 15)
ax.legend()

ax = fig.add_subplot(5, 1, 2)
v = (C16[0] * U_dft[:, 0]).real / N
ax.plot(v, '-o', color='blue', label=f'c0*u0')
ax.axhline(0, color='gray')
ax.set_ylim(-10, 15)
ax.legend()

for k in range(1, 4):
    ax = fig.add_subplot(5, 1, k+2)
    # c_k * u_k + c_{N-k} * u_{N-k} の計算
    v = (C16[k] * U_dft[:, k] + C16[N-k] * U_dft[:, N-k]).real / N
    ax.plot(v, '-o', color='blue', label=f'c{k}*u{k} + c{N-k}*u{N-k}')
    ax.axhline(0, color='gray')
    ax.set_ylim(-10, 15)
    ax.legend()

plt.show()

一番上のグラフは，$\mathbf{x}$ と $\mathbf{z}$ を重ねて描いたものです．$\mathbf{z}$ は，$\mathbf{x}$ を「定数」+「周期 $16$ の波」 + 「周期 $16/2$ の波」 + 「周期 $16/3$ の波」のみで近似したものなので，より周期の短い成分が失われていることが分かります．

2つ目以降には，$c_0\mathbf{u}_0$, $c_1\mathbf{u}_1 + c_{N-1}\mathbf{u}_{N-1}$, $c_2\mathbf{u}_2 + c_{N-2}\mathbf{u}_{N-2}$, $c_3\mathbf{u}_3 + c_{N-3}\mathbf{u}_{N-3}$ の 4つが描いてあります．例えば $c_1\mathbf{u}_1 + c_{N-1}\mathbf{u}_{N-1}$ は，$\mathbf{u}$ の実部が表す波の振幅を $2|c_1|$ 倍して，位相を $\arg c_1$ だけずらしたものになっています（上下に引き伸ばされ，左右にずれている）．

---
### スペクトル

上述のように，離散フーリエ変換では，与えられたベクトルを

- 定数成分
- 周期 $N$，振幅 $|c_1|$，位相 $\arg c_1$ の波
- 周期 $N/2$，振幅 $|c_2|$，位相 $\arg c_2$ の波
- 周期 $N/3$，振幅 $|c_3|$，位相 $\arg c_3$ の波
- ...

に分解して（これらの和として）表します．したがって，離散フーリエ変換の結果を分析する際には，得られたフーリエ係数 $c_k$ の値そのものよりも，その絶対値 $|c_k|$ と偏角 $\arg c_k$ を見る方が便利なことがよくあります．
横軸に成分の番号 $k$ をとり，縦軸に $|c_k|$ の値をプロットしたグラフを **振幅スペクトル** といい， $\arg c_k$ をプロットしたグラフを **位相スペクトル** といいます（注）．


※ 注意: 厳密な言い方をするとグラフではなくフーリエ係数の絶対値の集合 $\{ |c_k| \}$ を振幅スペクトル，偏角の集合 $\{ \arg c_k \}$ を位相スペクトルといいますが，ここではわかりやすさを重視して，それらのグラフを〇〇スペクトルと呼ぶと説明しています．



上のベクトルの例で振幅スペクトル，位相スペクトルを描いてみると，次のようになります．

In [ ]:
amplitude = np.abs(C16) # 絶対値
phase = np.angle(C16) # 偏角
print(amplitude)
print(phase)

# フーリエ係数の可視化（振幅と位相）
fig, ax = plt.subplots(1, 2, figsize=(12, 4))
ax[0].stem(amplitude, markerfmt='o', label='|Ck|')
ax[0].set_ylim(0, 60)
ax[0].legend()
ax[1].stem(phase, markerfmt='o', label='arg Ck')
ax[1].set_ylim(-3.5, 3.5)
ax[1].legend()
plt.show()

「準備」で読み込んだギターの音の場合，離散フーリエ変換して振幅スペクトルを描いてみると，次のようになります．

In [ ]:
# ギター音のデータを離散フーリエ変換して振幅スペクトルを求める
c = fft(guitar) # ここでは高速フーリエ変換を使用（あとで説明してます）
amplitude = np.abs(c)

# 振幅スペクトルを描く
fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(amplitude, label='|Ck|')
ax.legend()
plt.show()

離散フーリエ変換を適用して得られた展開係数の値をこのように可視化することで，データの性質をいろいろ分析することが可能となります．演習の方で実際の音のデータを分析してみる予定です．

---
### 高速フーリエ変換


コンピュータを用いて離散フーリエ変換の計算を実行する場合，原理的には，式 $(11)$ の計算を実行すればよいだけです．例えば，C言語で離散フーリエ変換を実装すると，次のように書けます（複素数の計算は実部と虚部に分けて行います）．

```
#define N  // データ点の数

  double x[N];       // 与えられるデータ
  double A[N], B[N]; // フーリエ係数の実部と虚部
  double omega0 = 2*M_PI/(double)N;

  // ここで x に値を代入

  /***** DFT *****/
  for(k = 0; k < N; k++){
    A[k] = B[k] = 0;
    for(n = 0; n < N; n++){
      A[k] += x[n]*cos(omega0*(double)(k*n)); // 実部
      B[k] -= x[n]*sin(omega0*(double)(k*n)); // 虚部
    }
    A[k] /= (double)N; B[k] /= (double)N;
  }
```

見ての通り，三角関数の計算と単純な積や和の計算を繰り返す2重ループです．この構造から想像がつく通り，離散フーリエ変換の計算量は，$N$ の2乗のオーダー（ $O(N^2)$ ）となります．$N$ が $10$倍，$100$倍，...，と大きくなると，計算量は $10^2 = 100$倍，$100^2 = 1$万倍，...，と大きくなっていくということです．

離散フーリエ変換を使う場面では $N$ の大きなデータを扱うことが多いですので，定義通りに計算する上記のようなやり方では計算量が大きすぎます．そのため実用的には，**高速フーリエ変換** (Fast Fourier Transform, FFT) という，離散フーリエ変換の高速演算アルゴリズムが使われます．
高速フーリエ変換のアルゴリズムを用いると，離散フーリエ変換を $O(n\log{n})$ の計算量で実行することができます．

前回も使っていた [SciPy](https://scipy.org/) の関数 [`scipy.fft.fft`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fft.fft.html) は，その名前の通り，高速フーリエ変換のアルゴリズムによって離散フーリエ変換を実行するものです．

高速フーリエ変換のアルゴリズムについては，この授業では説明を省略します．高校数学レベルの複素数の知識があれば理解できますので，興味があれば調べてみるとよいです．

高速フーリエ変換のありがたさを体感するために，ちょっと実験してみましょう．

「準備」で読み込んだギター音のデータは，次のようなものです：
- 標本化周波数 8000 [Hz]
- データ数 N = 16384
- 長さ N / 8000 = 2.048 [s]


In [ ]:
N = len(guitar)
print(N)

まずはこのデータの離散フーリエ変換を定義通りに計算した場合の時間を測ってみましょう．

In [ ]:
# 基底を求めておく
U = fft(np.eye(N))

次のセルを実行すると，実行にかかった時間が表示されます．

In [ ]:
%%time
# 離散フーリエ変換を定義通り計算
c = np.conj(U.T) @ guitar
print(c[1])

`CPU times:` の `user` の項が，上記の計算のために CPU が働いた時間です．実行のたびに少し変動しますが，1秒以上かかる結果となるでしょう．
このデータは約2秒の信号ですので，これではリアルタイムの処理（例えば，入力された音をその場でDFT → スペクトルをいじる → IDFTするような処理）は難しいですね．

では，高速フーリエ変換の方は...

In [ ]:
%%timeit -r 10 -n 100
# 高速フーリエ変換（ scipy.fft.fft ）
c = fft(guitar)
#print(c[1])

:速すぎるのでここでは 1000 回実行していますが，1回あたり200マイクロ秒 = 0.2ミリ秒（= $\frac{0.2}{1000}$ = $\frac{1}{5000}$秒）以下で済んでいます．

---
### ［よだんだよん］標本化定理


「標本化」について学んで分かったように，標本化周波数が低すぎると（標本化間隔が大きすぎると），元の信号の情報をうまく表せないかもしれません．かといって標本化周波数を高く（標本化間隔を小さく）すると，信号を表すための値の数が多くなってその後の処理が大変そうです．元の信号の情報をちゃんと表せるぎりぎりの標本化周波数をねらうとしたら，どのように決めたらよいでしょうか．これに関して，次の定理が知られています．


**標本化定理**

信号に含まれる有効な成分の中で最高の周波数が $f$ である場合，標本化周波数をその2倍すなわち $2f$ より高くして標本化すれば，標本化した信号から元の信号を再現できる．

このことは，直感的には次のように説明できます．

> ほげお: 信号を標本化したいけど，あまり標本化間隔を大きくとるとまずいみたい．かといって間隔を小さくするとデータ量が多くなって大変．標本化間隔はどうやって決めたらいい?
>
> ふがよ: 周期 $1$[s] （周波数 $1$ [Hz]） の正弦波の場合，標本化間隔が $0.5$ [s] より小さければ（標本化周波数が $2$ [Hz] より高ければ）よさそう．だから，標本化間隔は元の信号の周期の半分より小さくすれば（標本化周波数は元の信号の周波数の2倍より高くすれば）ええんちゃう?
>
> ほげお: でも正弦波じゃない一般の信号のときは?
>
> ふがよ: むー
>
> へなこ: フーリエ級数展開（フーリエ変換）考えたらどんな信号も正弦波の重ね合わせなんやから，その中でいちばん周期の短い（周波数の高い）正弦波が再現できるようにしたら大丈夫ちゃう？
>
> ほげお&ふがよ: おおー!